# VM 部署 Phi-4 模型完整教程

本 Jupyter Notebook 提供了在虚拟机（VM）上部署 Phi-4 模型的完整指南。本教程涵盖了从环境配置到模型部署和测试的所有步骤。

## 目录
1. [概述](#概述)
2. [环境要求](#环境要求)
3. [VM 环境准备](#vm-环境准备)
4. [安装依赖](#安装依赖)
5. [下载模型](#下载模型)
6. [部署模型](#部署模型)
7. [测试验证](#测试验证)
8. [性能优化](#性能优化)
9. [常见问题](#常见问题)

## 概述

Phi-4 是微软发布的一个高效的小型语言模型，本教程将指导您如何在虚拟机（VM）上完整部署 Phi-4 模型，包括环境配置、模型下载、部署和测试等步骤。

## 环境要求

### 硬件要求
- **CPU**: 至少 8 核心（推荐 16 核心或以上）
- **内存**: 最少 16GB RAM（推荐 32GB 或以上）
- **存储**: 至少 50GB 可用空间
- **GPU**: 可选，但强烈推荐使用 NVIDIA GPU（至少 8GB 显存）

### 软件要求
- **操作系统**: Ubuntu 20.04/22.04 LTS 或 CentOS 7/8
- **Python**: 3.8 或更高版本
- **CUDA**: 11.8 或更高版本（如果使用 GPU）

## 环境准备

### 定义环境变量

我们需要设置一些环境变量来管理我们的资源：

In [ ]:
import os
import random

# 设置环境变量
os.environ["RESOURCE_GROUP"] = "phi4-rg"
os.environ["LOCATION"] = "eastus3"
os.environ['VM_NAME'] = "phi4-vm"
os.environ['VM_SIZE'] = "Standard_NC6s_v3"
os.environ["COMPUTE_NAME"] = "phi4-k8s-compute"
os.environ["STORAGE_ACCOUNT"] = f"phi4storage{random.randint(0, 999999)}"

# 显示配置
print(f"Resource Group: {os.environ['RESOURCE_GROUP']}")
print(f"Location: {os.environ['LOCATION']}")
print(f"AKS Cluster: {os.environ['AKS_CLUSTER_NAME']}")
print(f"ML Workspace: {os.environ['ML_WORKSPACE_NAME']}")

## VM 环境准备

### 1. 创建资源组

以下使用 Azure CLI 创建 VM（需要先安装和配置 Azure CLI）：

In [ ]:
%%bash
az group create \
    --name $RESOURCE_GROUP \
    --location $LOCATION

### 2. 创建虚拟机

以下使用 Azure CLI 创建 VM（需要先安装和配置 Azure CLI）：

In [1]:
# 使用 Azure CLI 创建 VM
!az vm create \
  --resource-group $RESOURCE_GROUP \
  --name $VM_NAME \
  --image Ubuntu2204 \
  --size $VM_SIZE \
  --admin-username azureuser \
  --generate-ssh-keys

ERROR: (ResourceGroupNotFound) Resource group 'test-vm-rg' could not be found.
Code: ResourceGroupNotFound
Message: Resource group 'test-vm-rg' could not be found.


### 2. 连接到 VM

获取 VM 的 IP 地址后，使用 SSH 连接：

In [ ]:
# 替换为您的 VM IP 地址
VM_IP="<VM-IP-ADDRESS>"
!ssh azureuser@$VM_IP

### 3. 更新系统

运行以下命令更新系统并安装基础工具：

In [ ]:
# 更新包列表和系统
!sudo apt update && sudo apt upgrade -y

# 安装基础工具
!sudo apt install -y build-essential git wget curl

## 安装依赖

### 1. 安装 Python 和 pip

In [ ]:
# 安装 Python 3.10
!sudo apt install -y python3.10 python3.10-venv python3.10-dev python3-pip

# 创建 Python 虚拟环境
!python3.10 -m venv phi4-env
!source phi4-env/bin/activate

# 升级 pip
%pip install --upgrade pip

### 2. 安装 CUDA（如果使用 GPU）

In [ ]:
# 下载并安装 CUDA 12.1
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-keyring_1.1-1_all.deb
!sudo dpkg -i cuda-keyring_1.1-1_all.deb
!sudo apt-get update
!sudo apt-get -y install cuda-12-1

# 添加 CUDA 到环境变量
%%bash
echo 'export PATH=/usr/local/cuda-12.1/bin:$PATH' >> ~/.bashrc
echo 'export LD_LIBRARY_PATH=/usr/local/cuda-12.1/lib64:$LD_LIBRARY_PATH' >> ~/.bashrc
source ~/.bashrc

# 验证 CUDA 安装
!nvcc --version
!nvidia-smi

### 3. 安装 PyTorch 和相关库

In [ ]:
# 安装 PyTorch（GPU 版本）
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# 安装 Transformers 和相关库
%pip install transformers accelerate bitsandbytes
%pip install sentencepiece protobuf
%pip install gradio fastapi uvicorn

## 下载模型

### 1. 使用 Hugging Face CLI

In [ ]:
# 安装 Hugging Face CLI
%pip install huggingface-hub

# 登录 Hugging Face（需要账号）
!huggingface-cli login

# 创建模型目录
!mkdir -p ~/models/phi-4

### 2. 使用 Python 脚本下载

创建下载脚本 `download_phi4.py`：

In [ ]:
%%writefile download_phi4.py
from huggingface_hub import snapshot_download
import os

# 设置模型保存路径
model_path = os.path.expanduser("~/models/phi-4")

# 下载模型
snapshot_download(
    repo_id="microsoft/Phi-4",
    local_dir=model_path,
    cache_dir=model_path,
    local_dir_use_symlinks=False
)

print(f"模型已下载到: {model_path}")

In [ ]:
# 运行下载脚本
!python download_phi4.py

## 部署模型

### 1. 创建模型加载脚本

创建 `phi4_server.py`：

In [ ]:
%%writefile phi4_server.py
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import uvicorn
import logging

# 配置日志
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# 初始化 FastAPI
app = FastAPI(title="Phi-4 Model API")

# 定义请求和响应模型
class GenerationRequest(BaseModel):
    prompt: str
    max_length: int = 512
    temperature: float = 0.7
    top_p: float = 0.9
    top_k: int = 50

class GenerationResponse(BaseModel):
    generated_text: str
    prompt: str

# 全局变量存储模型和分词器
model = None
tokenizer = None

def load_model():
    """加载 Phi-4 模型"""
    global model, tokenizer
    
    model_path = "/home/azureuser/models/phi-4"
    
    logger.info("正在加载模型...")
    
    # 加载分词器
    tokenizer = AutoTokenizer.from_pretrained(
        model_path,
        trust_remote_code=True
    )
    
    # 检测可用设备
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    logger.info(f"使用设备: {device}")
    
    # 加载模型
    if torch.cuda.is_available():
        # GPU 加载，使用半精度
        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            trust_remote_code=True,
            torch_dtype=torch.float16,
            device_map="auto"
        )
    else:
        # CPU 加载
        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            trust_remote_code=True,
            torch_dtype=torch.float32
        )
        model = model.to(device)
    
    model.eval()
    logger.info("模型加载完成！")

@app.on_event("startup")
async def startup_event():
    """启动时加载模型"""
    load_model()

@app.get("/")
async def root():
    """健康检查端点"""
    return {"message": "Phi-4 Model API is running"}

@app.post("/generate", response_model=GenerationResponse)
async def generate_text(request: GenerationRequest):
    """生成文本端点"""
    try:
        # 编码输入
        inputs = tokenizer(
            request.prompt,
            return_tensors="pt",
            truncation=True,
            max_length=512
        )
        
        # 移动到正确的设备
        device = next(model.parameters()).device
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        # 生成文本
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_length=request.max_length,
                temperature=request.temperature,
                top_p=request.top_p,
                top_k=request.top_k,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id
            )
        
        # 解码输出
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        return GenerationResponse(
            generated_text=generated_text,
            prompt=request.prompt
        )
    
    except Exception as e:
        logger.error(f"生成文本时出错: {str(e)}")
        raise HTTPException(status_code=500, detail=str(e))

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

### 2. 创建 Gradio Web UI（可选）

创建 `phi4_gradio.py`：

In [ ]:
%%writefile phi4_gradio.py
import gradio as gr
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# 加载模型
def load_model():
    model_path = "/home/azureuser/models/phi-4"
    
    logger.info("正在加载 Phi-4 模型...")
    
    tokenizer = AutoTokenizer.from_pretrained(
        model_path,
        trust_remote_code=True
    )
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    if torch.cuda.is_available():
        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            trust_remote_code=True,
            torch_dtype=torch.float16,
            device_map="auto"
        )
    else:
        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            trust_remote_code=True,
            torch_dtype=torch.float32
        )
        model = model.to(device)
    
    model.eval()
    logger.info("模型加载完成！")
    
    return model, tokenizer, device

# 生成函数
def generate_response(prompt, max_length, temperature, top_p):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# 加载模型
model, tokenizer, device = load_model()

# 创建 Gradio 界面
iface = gr.Interface(
    fn=generate_response,
    inputs=[
        gr.Textbox(
            lines=5,
            label="输入提示词",
            placeholder="请输入您的问题或提示..."
        ),
        gr.Slider(
            minimum=50,
            maximum=1024,
            value=256,
            step=50,
            label="最大长度"
        ),
        gr.Slider(
            minimum=0.1,
            maximum=2.0,
            value=0.7,
            step=0.1,
            label="温度"
        ),
        gr.Slider(
            minimum=0.1,
            maximum=1.0,
            value=0.9,
            step=0.05,
            label="Top-p"
        )
    ],
    outputs=gr.Textbox(
        lines=10,
        label="生成的文本"
    ),
    title="Phi-4 模型演示",
    description="使用 Phi-4 模型生成文本",
    examples=[
        ["请解释什么是人工智能？", 256, 0.7, 0.9],
        ["写一首关于春天的诗", 256, 0.9, 0.95],
        ["如何学习编程？", 512, 0.7, 0.9]
    ]
)

if __name__ == "__main__":
    iface.launch(server_name="0.0.0.0", server_port=7860)

### 3. 创建系统服务（可选）

创建 systemd 服务文件 `/etc/systemd/system/phi4.service`：

In [ ]:
%%writefile /etc/systemd/system/phi4.service
[Unit]
Description=Phi-4 Model Service
After=network.target

[Service]
Type=simple
User=azureuser
WorkingDirectory=/home/azureuser
Environment="PATH=/home/azureuser/phi4-env/bin"
ExecStart=/home/azureuser/phi4-env/bin/python /home/azureuser/phi4_server.py
Restart=on-failure
RestartSec=10

[Install]
WantedBy=multi-user.target

In [ ]:
# 启动服务
!sudo systemctl daemon-reload
!sudo systemctl enable phi4.service
!sudo systemctl start phi4.service
!sudo systemctl status phi4.service

## 测试验证

### 1. 测试 API 端点

In [ ]:
# 测试健康检查
!curl http://localhost:8000/

# 测试文本生成
%%bash
curl -X POST "http://localhost:8000/generate" \
     -H "Content-Type: application/json" \
     -d '{
       "prompt": "什么是人工智能？",
       "max_length": 256,
       "temperature": 0.7
     }'

### 2. Python 客户端测试

创建 `test_client.py`：

In [ ]:
%%writefile test_client.py
import requests
import json

# API 端点
url = "http://localhost:8000/generate"

# 测试用例
test_prompts = [
    "请解释量子计算的基本原理",
    "写一个 Python 函数计算斐波那契数列",
    "描述一下未来的智能城市"
]

for prompt in test_prompts:
    print(f"\n提示词: {prompt}")
    print("-" * 50)
    
    response = requests.post(
        url,
        json={
            "prompt": prompt,
            "max_length": 256,
            "temperature": 0.7
        }
    )
    
    if response.status_code == 200:
        result = response.json()
        print(result["generated_text"])
    else:
        print(f"错误: {response.status_code}")

In [ ]:
# 运行测试客户端
!python test_client.py

## 性能优化

### 1. 量化优化

使用 bitsandbytes 进行 8-bit 量化：

In [ ]:
%%writefile quantization_example.py
from transformers import BitsAndBytesConfig

# 配置 8-bit 量化
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.float16
)

# 加载量化模型
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=quantization_config,
    device_map="auto",
    trust_remote_code=True
)

### 2. 批处理优化

In [ ]:
%%writefile batch_processing.py
def batch_generate(prompts, batch_size=4):
    """批量生成文本"""
    results = []
    
    for i in range(0, len(prompts), batch_size):
        batch = prompts[i:i+batch_size]
        inputs = tokenizer(
            batch,
            return_tensors="pt",
            padding=True,
            truncation=True
        )
        
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_length=256,
                temperature=0.7,
                do_sample=True
            )
        
        for output in outputs:
            text = tokenizer.decode(output, skip_special_tokens=True)
            results.append(text)
    
    return results

### 3. 缓存优化

In [ ]:
%%writefile caching.py
from functools import lru_cache
import hashlib

@lru_cache(maxsize=1000)
def cached_generate(prompt_hash, max_length, temperature):
    """缓存生成结果"""
    # 实际生成逻辑
    pass

def generate_with_cache(prompt, **kwargs):
    """带缓存的生成函数"""
    prompt_hash = hashlib.md5(prompt.encode()).hexdigest()
    return cached_generate(prompt_hash, **kwargs)

## 常见问题

### 1. 内存不足

In [ ]:
# 使用 CPU offload
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    offload_folder="offload",
    offload_state_dict=True
)

### 2. GPU 驱动问题

In [ ]:
# 重新安装 NVIDIA 驱动
!sudo apt-get purge nvidia-*
!sudo apt-get autoremove
!sudo apt-get update
!sudo apt-get install nvidia-driver-525
!sudo reboot

### 3. 模型下载失败

In [ ]:
# 使用镜像加速
!export HF_ENDPOINT=https://hf-mirror.com

# 或使用代理
!export https_proxy=http://your-proxy:port
!export http_proxy=http://your-proxy:port